<a href="https://colab.research.google.com/github/dnguyend/rayleigh_newton/blob/master/colab/BPairsHomogeneousSurface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\cT}{\mathcal{T}}$
$\newcommand{\hcT}{\hat{\mathcal{T}}}$
$\newcommand{\cB}{\mathcal{B}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\R}{\mathbb{R}}$
# This workbook shows example of a BPair using the corresponding hypersurface constraint. Assume $m\neq d$

* Eigenpairs  
$$\cT(X^{[m-1]}) =\lambda \cB(X^{[d-1]}) $$
$T(X[m−1])$  is a vector-valued function, from  $\R^n$  to itself, each entry is homogeneous of order  $m−1$ . We also use the notation  $T(I,X[m−1])=T(X[m−1])$  The square bracket means the number of times  $X$  is repeated. Similarly
$\cB(X[m−1])$ is a vector-valued function with entries homogeneous polynomials of order $d-1$.

* Use generalized Rayleigh quotient method.

* Assume $X\in \R^n$, consider the constraints $\cB(X^{[d]}) = 1$. This is not the most general case, we could have eigenpairs with $\cB(X^{[d]}) = -1$ or $\cB(X^{[d]}) = 0$, but we consider this case for simplicity. The left inverse is $ X^T$, the Rayleigh quotient is 
$$\lambda = \cT(X^{[m-1]})^TX.$$
* if $\cT$ is symmetric, $\hcT(X^{[m]}):= \cT(X^{[m-1]})^TX$ is scalar, homogeneous of order $m$ and $\hcT' = m\cT$.

The eigenvalue count is 
$$\frac{(m-1)^n - (d-1)^n}{m - d} = \sum_{i=0}^{m-1}(m-1)^i(d-1)^{n-1-i}
$$

* Some cells may be hidden. Please download and open in colab, then expand the hidden cells
First, clone the project from github


In [1]:
!git clone https://github.com/dnguyend/rayleigh_newton

Cloning into 'rayleigh_newton'...
remote: Enumerating objects: 280, done.
remote: Counting objects: 100% (280/280), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 280 (delta 145), reused 258 (delta 132), pack-reused 0
Receiving objects: 100% (280/280), 14.81 MiB | 26.70 MiB/s, done.
Resolving deltas: 100% (145/145), done.


Importing main functions to be used later - but the code to find all complex eigen pairs is in the next block.

In [2]:
import numpy as np
import numpy.linalg as la
from rayleigh_newton.core import utils as ut
from IPython.core.display import display
from rayleigh_newton.core import bne_real_surface as bnr
from rayleigh_newton.core import bne_eigen_tensor_solver as bne

First, the real case with constraint $B(X^{[d]}) = \epsilon$.
The tangent space consisting of $\eta$ with $B(\eta, X^{[d-1]}) = 0$. For this, we create a class Bfeasible to model the feasible set. We include a retraction, sending a pair $(X, \eta)$ to a point $\gamma(X+\eta)$ on the feasible set with $\gamma > 0$.

The Rayleigh quotient is $\lambda= R(X)= \epsilon T(X^{[m]})$.



Test this class: retraction and its derivatives

In [3]:
n = 5
m = 4
d = 3

T = ut.generate_symmetric_tensor(n, m)
B = ut.generate_symmetric_tensor(n, d)

Bf = bnr.Bfeasible(B)
x = Bf.rand()
eta = Bf.randvec(x)
print("Tangent vector is normal to B(x^{d-1}")
display(np.sum(ut.tv_mode_product(B, x, modes=d-1)*eta))

x1 = Bf.tensor_rtr(x, eta)
print("check that retraction gives a point on the constrained surface B(x^d) = 1")
display(ut.tv_mode_product(B, x, modes=d))

# test D_tensor_rtr:
dlt = 1e-7
eta1 = Bf.randvec(x)

print("test D_tensor_rtr:  derivative of Rtr is same as numerical derivative:")
print((Bf.tensor_rtr(x, eta + dlt*eta1) - Bf.tensor_rtr(x, eta))/dlt
      - Bf.D_tensor_rtr(x, eta, eta1))
print("test H_tensor_rtr:  H_tensor_rtr is same as 2nd numerical derivative:")    
print((Bf.D_tensor_rtr(x, dlt*eta, eta) - Bf.D_tensor_rtr(x, 0, eta))/dlt
      - Bf.H_tensor_rtr(x, eta))

Tangent vector is normal to B(x^{d-1}


-3.3306690738754696e-16

check that retraction gives a point on the constrained surface B(x^d) = 1


array(1.)

test D_tensor_rtr:  derivative of Rtr is same as numerical derivative:
[-1.04442687e-08  2.68265884e-08  5.77132842e-10  1.92579958e-08
  2.14047011e-08]
test H_tensor_rtr:  H_tensor_rtr is same as 2nd numerical derivative:
[ 1.38343822e-07 -1.17522357e-06 -4.24282541e-07  7.58985151e-07
 -1.03112523e-08]


# The Schur form iteration for the B tensor eigenpair

Test run for the $B$ tensor pair. Showing at a solution various components of the Chebyshev term is zero.

The only non zero term is $L_{XX}$

In [4]:
n = 5
m = 4
d = 3
# np.random.seed(0)
T = ut.generate_symmetric_tensor(n, m)
B = ut.generate_symmetric_tensor(n, d)
max_itr = 200
Bf = bnr.Bfeasible(B)
ret = bnr.schur_form_B_tensor_rayleigh_chebyshev(
        T, Bf, max_itr, delta=1e-8, x_init=None, do_chebyshev=False)
print('doing RQI')
print(ret)
ret = bnr.schur_form_B_tensor_rayleigh_chebyshev(
        T, Bf, max_itr, delta=1e-8, x_init=None, do_chebyshev=True)
print('doing Rayleigh-Chebyshev')
print(ret)

x = ret[0]

lbd = ut.tv_mode_product(T, x, m)
# show various components are zeros

print('Derivative of Rayleigh quotient at x_* = %.5f' %
      bnr.Rayp(T, Bf, x, Bf.randvec(x)))

eta = Bf.randvec(x)
print('Projection of Lx Hessian of retraction =%s' %
      str(Bf.Pi(x, bnr.Lx(T, Bf, x, lbd)@Bf.H_tensor_rtr(x, eta))))
print('Projection of L_lambda=%s' %
      str(Bf.Pi(x, bnr.tv_mode_product(Bf.B, x, Bf.d-1))))


doing RQI
(array([0.2876865 , 0.30895075, 0.02789629, 0.49697842, 0.22450467]), 1.5129424422810336, 17, True, 5.438959822042073e-16)
doing Rayleigh-Chebyshev
(array([ 0.37588416,  0.49867558, -0.13448741, -0.55772844,  0.2387314 ]), -0.1295293405973719, 11, True, 1.0126982252310917e-16)
Derivative of Rayleigh quotient at x_* = -0.00000
Projection of Lx Hessian of retraction =[-9.99200722e-16 -1.11022302e-15 -3.33066907e-16 -1.22124533e-15
  3.88578059e-16]
Projection of L_lambda=[0. 0. 0. 0. 0.]


This confirms the analysis in the paper.